- Load a dataset

- Make it as a iterator

- Load a tokeniser

- Write a processing function 

- Map it to the dataset 

- Create a new tokeniser

- Train it with the dataset

- Write the post processing function

- Run the evaluation 

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [3]:
imdb_ds = load_dataset("imdb")

In [4]:
imdb_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
model_name = "distilbert-base-uncased"
model_path = "/home/kamal/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [6]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [6]:
tokenizer_from_name = AutoTokenizer.from_pretrained(model_name)

In [7]:
tokenizer_from_name.model_max_length

512

In [7]:
def tokenise_ds(row):
    return tokenizer_from_name(row['text'], padding=True,
                               truncation=True, 
                               max_length=tokenizer_from_name.model_max_length,
                              return_tensors='pt')

In [8]:
tokenise_imdb = imdb_ds.map(tokenise_ds, batched=True, num_proc=3)

In [17]:
from rich import print

print(tokenise_imdb['train'][0])

{
    'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it 
when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to 
enter this country, therefore being a fan of films considered "controversial" I really had to see this for 
myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn 
everything she can about life. In particular she wants to focus her attentions to making some sort of documentary 
on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the 
United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, 
she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW
is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, 
even then it\'s not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex 
and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John 
Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the 
film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic 
theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun 
intended) of Swedish cinema. But really, this film doesn\'t have much of a plot.',
    'label': 0,
    'input_ids': [
        101,
        1045,
        12524,
        1045,
        2572,
        8025,
        1011,
        3756,
        2013,
        2026,
        2678,
        3573,
        2138,
        1997,
        2035,
        1996,
        6704,
        2008,
        5129,
        2009,
        2043,
        2009,
        2001,
        2034,
        2207,
        1999,
        3476,
        1012,
        1045,
        2036,
        2657,
        2008,
        2012,
        2034,
        2009,
        2001,
        8243,
        2011,
        1057,
        1012,
        1055,
        1012,
        8205,
        2065,
        2009,
        2412,
        2699,
        2000,
        4607,
        2023,
        2406,
        1010,
        3568,
        2108,
        1037,
        5470,
        1997,
        3152,
        2641,
        1000,
        6801,
        1000,
        1045,
        2428,
        2018,
        2000,
        2156,
        2023,
        2005,
        2870,
        1012,
        1026,
        7987,
        1013,
        1028,
        1026,
        7987,
        1013,
        1028,
        1996,
        5436,
        2003,
        8857,
        2105,
        1037,
        2402,
        4467,
        3689,
        3076,
        2315,
        14229,
        2040,
        4122,
        2000,
        4553,
        2673,
        2016,
        2064,
        2055,
        2166,
        1012,
        1999,
        3327,
        2016,
        4122,
        2000,
        3579,
        2014,
        3086,
        2015,
        2000,
        2437,
        2070,
        4066,
        1997,
        4516,
        2006,
        2054,
        1996,
        2779,
        25430,
        14728,
        2245,
        2055,
        3056,
        2576,
        3314,
        2107,
        2004,
        1996,
        5148,
        2162,
        1998,
        2679,
        3314,
        1999,
        1996,
        2142,
        2163,
        1012,
        1999,
        2090,
        4851,
        8801,
        1998,
        6623,
        7939,
        4697,
        3619,
        1997,
        8947,
        2055,
        2037,
        10740,
        2006,
        4331,
        1010,
        2016,
        2038,
        3348,
        2007,
        2014,
        3689,
      

In [16]:
import evaluate

accuracy = evaluate.load('accuracy')

In [23]:
import numpy as np
def compute_accuracy(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return accuracy.compute(predictions=preds, references=labels)

In [11]:
id2label = {0:"negative", 1:"positive"}
label2id = {"negative":0, "positive":1}

In [19]:
import torch
del model_wt
torch.cuda.empty_cache()

In [12]:
model_wt = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=model_path,
    num_labels=2,
    id2label=id2label,
    label2id= label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /home/kamal/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/ and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
moved = model_wt.to('cuda')

In [24]:
targs = TrainingArguments(
    output_dir='/home/kamal/training_files/distil_imbd',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy='steps',
    save_strategy='steps',
    save_steps=1500,
    eval_steps=500,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

In [25]:
trainer = Trainer(
    model=model_wt,
    args=targs,
    train_dataset=tokenise_imdb['train'],
    eval_dataset=tokenise_imdb['test'],
    compute_metrics=compute_accuracy
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.064100,0.393557,0.916160
1000,0.143700,0.412702,0.904400
1500,0.116000,0.311612,0.925200
2000,0.098300,0.319082,0.928120
2500,0.096400,0.291300,0.928880
3000,0.084700,0.314138,0.930200


TrainOutput(global_step=3126, training_loss=0.10026636386024441, metrics={'train_runtime': 1485.889, 'train_samples_per_second': 33.65, 'train_steps_per_second': 2.104, 'total_flos': 6623369932800000.0, 'train_loss': 0.10026636386024441, 'epoch': 2.0})

In [27]:
trained_model_path = "/home/kamal/training_files/distil_imbd/checkpoint-3000/"
trained_model = AutoModelForSequenceClassification.from_pretrained(trained_model_path)

In [48]:
trained_model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [49]:
test = "The training process was a good experience, and took a lot of time"
tokened_test = tokenizer_from_name(test, return_tensors='pt').to('cuda')
tokened_test

{'input_ids': tensor([[ 101, 1996, 2731, 2832, 2001, 1037, 2204, 3325, 1010, 1998, 2165, 1037,
         2843, 1997, 2051,  102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [50]:
model_out = trained_model(**tokened_test).logits

In [51]:
prediction = model_out.argmax().item()
trained_model.config.id2label[prediction]

'positive'

In [42]:
imdb_ds['unsupervised'][10]

{'text': "This isn't the worst comedy of all-time, but that is about the best thing that I can say about this pathetic film. I didn't laugh once, or even smile once during this bomb. There was usually something going on on-screen, so I didn't get TOO bored, but most of the jokes here were simply awful. The final sequence is nothing more than a long series of people falling through doors and stumbling all over the place. Needless to say, it was a fitting way to end a movie that was impossible for me to like.",
 'label': -1}

In [52]:
def get_sentiment(row):
    text = row['text']
    tokened_text = tokenizer_from_name(text, truncation=True,
                                      padding=True, max_length=512,
                                      return_tensors='pt').to('cuda')
    model_out = trained_model(**tokened_text).logits
    prediction = model_out.argmax().item()
    return {'prediction': trained_model.config.id2label[prediction]}

In [45]:
get_sentiment(imdb_ds['unsupervised'][0])

{'prediction': 'positive'}

In [53]:
predicting_imdb = imdb_ds['unsupervised'].map(get_sentiment)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [58]:
predicting_imdb.save_to_disk(dataset_path='~/.cache/huggingface/datasets/imdb_unsupervised_predicted')

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]